In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math

import sklearn
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.utils import resample
from sklearn.tree import plot_tree
from sklearn import metrics
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
base_mod = pd.read_csv('https://raw.githubusercontent.com/MadavR/MIAD-VyS/main/EXA_2022_1_Todos_Limp.csv', sep=',')

In [3]:
# En el escenario 8 intentamos predecir el resultado de Ciencias Naturales
# Consideramos todas las variables categóricas y numéricas
# Además, consideramos la categorización de los resultados: 0 (o-4 respuestas correctas), 1 (5-9 respuestas correctas),
# 2 (10-14 respuestas correctas) y 3 (15-20 respuestas correctas)

In [4]:
base_mod_cat = base_mod.assign(EXA_C_RTAS_CORR_CN = lambda x: ((base_mod['EXA_N_RTAS_CORR_CN'] // 5)))
base_mod_cat = base_mod_cat.assign(EXA_C_RTAS_CORR_CC = lambda x: ((base_mod['EXA_N_RTAS_CORR_CC'] // 5)))
base_mod_cat = base_mod_cat.assign(EXA_C_RTAS_CORR_LC = lambda x: ((base_mod['EXA_N_RTAS_CORR_LC'] // 5)))
base_mod_cat = base_mod_cat.assign(EXA_C_RTAS_CORR_MT = lambda x: ((base_mod['EXA_N_RTAS_CORR_MT'] // 5)))

In [5]:
base_mod_cat['EXA_C_RTAS_CORR_CN'] = base_mod_cat['EXA_C_RTAS_CORR_CN'].replace([4], 3)
base_mod_cat['EXA_C_RTAS_CORR_CC'] = base_mod_cat['EXA_C_RTAS_CORR_CC'].replace([4], 3)
base_mod_cat['EXA_C_RTAS_CORR_LC'] = base_mod_cat['EXA_C_RTAS_CORR_LC'].replace([4], 3)
base_mod_cat['EXA_C_RTAS_CORR_MT'] = base_mod_cat['EXA_C_RTAS_CORR_MT'].replace([4], 3)

In [ ]:
base_mod_cat.columns

Index(['Unnamed: 0', 'ESTU_CONSECUTIVO', 'ESTU_GRADO', 'ESTU_GENERO',
       'COLE_NOM_ESTABLECIMIENTO', 'COLE_COD_ICFES', 'COLE_NATURALEZA',
       'COLE_CALENDARIO', 'COLE_JORNADA', 'COLE_COD_MCPIO', 'COLE_NOM_MCPIO',
       'COLE_COD_DPTO', 'COLE_NOM_DPTO', 'EXA_MODALIDAD', 'EXA_N_RTAS_CORR_CN',
       'EXA_N_RTAS_CORR_CC', 'EXA_N_RTAS_CORR_LC', 'EXA_N_RTAS_CORR_MT',
       'EXA_C_RTAS_CORR_CN', 'EXA_C_RTAS_CORR_CC', 'EXA_C_RTAS_CORR_LC',
       'EXA_C_RTAS_CORR_MT'],
      dtype='object')

In [6]:
# Crea una lista de las variables categóricas
categoricas_var = ["ESTU_GENERO", "COLE_NATURALEZA", "COLE_JORNADA", "EXA_MODALIDAD"]

# Realiza la codificación one hot
for var in categoricas_var:
    base_mod_cat = pd.get_dummies(base_mod_cat, columns=[var])

In [7]:
X = base_mod_cat[['ESTU_GRADO',  'COLE_COD_ICFES', 'COLE_COD_MCPIO', 'COLE_COD_DPTO',
                    'EXA_C_RTAS_CORR_MT', 'EXA_C_RTAS_CORR_CC', 'EXA_C_RTAS_CORR_LC',
               'ESTU_GENERO_F', 'COLE_NATURALEZA_O',
               'COLE_JORNADA_COMPLETA', 'COLE_JORNADA_MAÑANA', 'COLE_JORNADA_TARDE',
                  'EXA_MODALIDAD_OFF', 'EXA_MODALIDAD_ON', 'EXA_MODALIDAD_PA']]

# Selecciona la variable dependiente
y = base_mod_cat['EXA_C_RTAS_CORR_CN']

In [8]:
# Divide los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Entrenar el algoritmo de XGBoost
model_XGB = XGBClassifier()
model_XGB.fit(X_train, y_train)

# Evalúa el modelo
score_XGB = model_XGB.score(X_test, y_test)
print(score_XGB)

prediccion_XGB = model_XGB.predict(X_test)
MSE_XGB = np.average(np.square(prediccion_XGB - y_test))
print("El MSE del modelo en la muestra de evaluación es igual a:")
print(MSE_XGB)

0.6918472063854048
El MSE del modelo en la muestra de evaluación es igual a:
0.41790193842645385


In [ ]:
# Definir el modelo a optimizar
model = XGBClassifier()

# Definir el espacio de parámetros a explorar
param_grid = {
    "n_estimators": [100, 200, 300],
    "learning_rate": [0.1, 0.2, 0.3],
    "max_depth": [40, 60, 80],
}

# Instanciar el GridSearchCV
gscv = GridSearchCV(model, param_grid, scoring="accuracy", n_jobs=-1)

# Entrenar el modelo
gscv.fit(X_train, y_train)

# Imprimir los resultados
print(gscv.best_params_)
print(gscv.best_score_)

{'learning_rate': 0.2, 'max_depth': 40, 'n_estimators': 100}
0.7097647897362794
